# Lab 7: Starting your Final Project!

## Name: 

---

Today we are going to take everything we have learned and start your final project!  Yesterday, we reviewed the fundamentals of data reduction, star clusters, and asking scientific questions.  For the next 2 weeks, we are going to take all those ideas and apply them to astronomical data in order to answer the questions you created!   

Yesterday, your group was assigned an open cluster.  Over the next week, you will learn about your cluster, and reduce the data, perform photometry on your data and then create a Color-Magnitude Diagram.  

It is highly recommend saving your notebooks + reduced data in a google drive, email, or another online storage location of your choice.  

The first steps to reducing and analysing any data is calibration!      

As you learned earlier, you will be reducing two targets -- your cluster and your standard star.  As you are in groups of 2 or 3, each member of your group will reduce data in 1 filter (either B, V, R, or I) which you will then combine with your groupmates in order to create your CMD.      

To calibrate and reduce our data, we are going to perform the following steps *over the next couple of days*:

1. Reduce our data using calibration biases, darks, and flats!
    
     $\mathrm{Final\ reduced\ image} = \frac{\mathrm{ science\ image} - \mathrm{calibration\ bias} -  \mathrm{calibration\ dark \ (at\ same\ exposure\ time\ as\ science\ image)}}{  \mathrm{calibration\ flat}}$
2. Align each individual image to each other.
3. Stack our individual images in each band!
4. Start analysis!


## Refresher: What are these "Final Calibration Images"?  

Why is this step so important?  Why are we creating final calibration frames?  And why would we combine many images into a single one?  

<img src="final_cal_image.png">

When we combine our images, we can remove any outliers that can contaminate a single image which would affect our science image if we tried to subtract that outlier.  Therefore, doing bias subtraction is very important to creating well calibrated data. 

# 7.1 Calibrate Science and Standard Data

We have provided you these final calibration images in their already reduced and ready to go format, so you can start reducing your data!

You have each been assigned a filter to reduce.  In the cell below, change **V** to the band you will be reducing: 
- B
- V
- R
- I


In [ ]:
band = 'V' #default is V, but be sure to change this if you are using a different band


Now, let's get the paths to all of our final calibration images.  

First is the main path to the folder where you data is located (i.e. the location where your data is stored).  Use the example in the comment in the cell below to create the path on your computer to your data.

In [ ]:
# Example:
# datadir = '/Users/sarah/Documents/Data/M52/'

datadir = '/Users/sarah/Desktop/NGC6811/' < - mac
datadir = 'C:\\Users\\sarah\\Desktop\\NGC6811\\' <- windows V1
datadir = 'C:/Users/sarah/Desktop/NGC6811/' <- windows V2


In [ ]:
bias_path = datadir + 'Cal/Bias.fit'
flat_path = datadir + 'Cal/Flat_'+band+'band.fit'
dark_path = datadir + 'Cal/Dark_60s.fit'

Now, lets work through the standard star data apply our calibrations to the data!  You will then work through the cluster data!  

In [ ]:
# The standard fare:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt

from astropy.io import fits
from astropy.visualization import ZScaleInterval
interval = ZScaleInterval()
import glob
import os

In [ ]:
# fill in the name of the folder with your standard star data
#(e.g.: std_star = 'standard_G3-33')

std_star = '??'

In [ ]:
# Get path and glob together our standard star frames

# fill in the ?? with the name of the standard star folder
std_datadir = datadir + ?? + '/'+band+'band/'

# glob all files in std_datadir
std_files = glob.glob(std_datadir + '*.fit')

print(band+' band standard star images')
print(np.array(std_files))

## 7.1.1 Bias Subtraction

Our first step is to subtract off the bias.  This is because every single image we take (science and calibration) has some bias (or extra positive counts which are added to each image).  We need to get rid of these extra counts in order to figure out how much light is actually coming from our astronomical objects, or from the flat frame.   
 

Now, let's subtract the bias from our standard star frames. Do this below for all of the files, using your `bias_subtract` function.


    

In [ ]:
def bias_subtract(filename, path_to_bias, outpath):
    
    print('bias subtracting: ',filename)
    # open up the flat field and/or science image
    frame_data = fits.getdata(filename)
    
    # open up the header information for the flat field and/or science image
    frame_header = fits.getheader(filename)
    
    # open up the master bias image
    master_bias_data = fits.getdata(path_to_bias)
    
    # subtract off the bias from the flat field and/or science image  
    bias_subtracted = frame_data - master_bias_data
    
    # new_filename is taking the name of your file: '/path/to/your/cluster/clusterfolder/calibration/darks/dark60s.fit')  
    # and splitting it up at every '/' and put it in a list: ['path', 'to', 'your', cluster', ... ,'darks', 'dark60s.fit']
    # and taking the last string in the list (e.g. new_filename = 'dark60s.fit').  
    new_filename = filename.split('/')[-1]
    
    print('bias subtracted image save as: ', datadir + outpath + 'b_' + new_filename)
    fits.writeto(datadir + outpath + 'b_' + new_filename, bias_subtracted, frame_header,overwrite=True)
    print()
    return 

<div class="alert alert-block alert-danger">
    
Go through the function above.  Try to write a short description of what the function is doing, using both the code and the comments.


**Answer:**

In [ ]:
# Now, we bias subtract all the standard star data using our final calibration bias. 

# We will first remove any existing bias subtracted frames.
b_stddata = glob.glob(datadir + std_star + '/'+band+'band/b*.fit*')
for im in b_stddata:
    os.remove(im)
    
# reglob files incase you deleted any b_ files
std_files = glob.glob(std_datadir + '*.fit')

#go through all the band frames and apply the final calibration bias
for fitsfile in std_files:
    bias_subtract(fitsfile, bias_path, std_star + '/'+band+'band/')

You should now have twice the number of standard star in the band directory, half of which have the prefix 'b_'. These are the frames we want to subtract out the dark current and and flat field.

<div class="alert alert-block alert-warning"> 
    
Check the folder with you standard star frames and make sure these files exist.

In [ ]:
# Glob your b_ standard star files

# glob all files in std_datadir
b_std_files = glob.glob(std_datadir + 'b_*.fit')

print(band+' band bias subtracted standard star images')
print(np.array(b_std_files))

## 7.1.2 Dark Subtraction

Now, we will want to subtract the dark contribution from the bias subtracted frames. This can be accomplished by creating a new function below, `dark_subtract`, that looks very much like your `bias_subtract` function. 

 * Most importantly, the final calibration dark you subtract must matches the exposure time of your science images! 

We will now dark subtract our bias subtracted files using the `dark_subtract` function below and the 60s final calibration dark.  We are going to scale our final calibration dark to match the exposure time of the standard stars. 

In [ ]:
# dark subtraction function here:

def dark_subtract(filename, path_to_dark, outpath):
    '''
    performs dark subtraction on your flat/science fields. 
    '''
    
    # open the flat/science field data and header
    frame_data = fits.getdata(filename)
    frame_header = fits.getheader(filename)
    
    #open the final calibration dark frame with the same exposure time as your data. 
    cal_dark_data = fits.getdata(path_to_dark)
    
    #subtract off the dark current 
    if frame_header['EXPTIME'] != fits.getheader(path_to_dark)['EXPTIME']:
        scale = frame_header['EXPTIME'] / fits.getheader(path_to_dark)['EXPTIME']
        cal_dark_data = scale * cal_dark_data
    
    dark_subtracted = frame_data - cal_dark_data
    
    new_filename = filename.split('/')[-1]
    
    print('dark subtracted image save as: ', datadir + outpath + 'd' + new_filename)
    fits.writeto(datadir + outpath + 'd' + new_filename, dark_subtracted, frame_header,overwrite=True)
    print()
    return 

<div class="alert alert-block alert-danger">
    
1. Go through the function above.  Try to write a short description of what the function is doing, using both the code and the comments.


**Answer:**
    
2. What exposure times are your standard star frames.  Will the function need to scale your final calibration dark?
    
**Answer:**

In [ ]:
# We will first remove any existing dark subtracted frames.
db_stddata = glob.glob(datadir + std_star + '/'+band+'band/db*.fit*')
for im in db_stddata:
    os.remove(im)

print('dark subtracting standard star '+band+'band')
for b_sci_image in b_std_files:

    dark_subtract(b_sci_image, dark_path, std_star + '/'+band+'band/')
    

When you save your dark-subtracted images, another prefix is added to the file name.  Also, it's important to only dark subtract the **already bias-subtracted images**. For example, your function should subtract the dark current from 'b_standardstar1.fit', and then make a new FITS file called 'db_stardardstar1.fit'.

## 7.1.3 Normalize by Flat Field

Finally, we flat field our standard star data by dividing out bias subtracted, dark subtracted frames with the normalized final calibration flat.

First, glob the bias and dark subtracted data frames.

In [ ]:
# Glob your db_ standard star files

# glob all files in std_datadir
db_std_files = glob.glob(std_datadir + 'db_*.fit')

print(band+' band bias/dark subtracted standard star images')
print(np.array(db_std_files))

In [ ]:
# We will first remove any existing flat normalized frames.
fdb_stddata = glob.glob(datadir + std_star + '/'+band+'band/fdb*.fit*')
for im in fdb_stddata:
    os.remove(im)


# flat field the db_ standard star frames.
print('flat fielding db_ standard star '+band+' band')
flat_data = fits.getdata(flat_path)
for db_sci_image in db_std_files:
    #open the data and header for the db_ image
    db_sci_data = fits.getdata(db_sci_image)
    db_sci_hdr = fits.getheader(db_sci_image)

    #divide the science image by the final calibration flat field
    fdb_sci_image = db_sci_data / flat_data
    
    #save the new flat fielded image!
    sci_name = db_sci_image.split('/')[-1]
    outpath = std_star + '/'+band+'band/'
    
    print('image saved to: ' + datadir + outpath + 'f' + sci_name)
    fits.writeto(datadir + outpath + 'f' + sci_name, fdb_sci_image, db_sci_hdr, overwrite=True )
    

## Congrats!  You have now calibrated your standard star data!  Let's take a look at it.

In [ ]:
# plot a raw standard star frame and a final calibrated standard star frame.

#glob together all the flat field calibrated standard star data
final_std_files = glob.glob(datadir + std_star + '/'+band+'band/fdb*.fit')

#Now, use fits.getdata to read in a raw standard star file.
# replace ?? with a 0.
raw_std_file = fits.getdata(std_files[??])

# replace ?? with a 0.
final_std_file = fits.getdata(final_std_files[??])


fig, (ax1, ax2) = plt.subplots(figsize=(15,4), nrows=1, ncols=2)
vmin, vmax = interval.get_limits(raw_std_file)
im1 = ax1.imshow(raw_std_file,vmin=vmin, vmax=vmax ) #plot raw standard star file
plt.colorbar(im1, ax=ax1)

vmin, vmax = interval.get_limits(final_std_file)
im2 = ax2.imshow(final_std_file, vmin=vmin, vmax=vmax) #plot calibrated standard star file
plt.colorbar(im2, ax=ax2)

#create a title, axis labels, change the colormap, to create a nice looking plot!

plt.show()

<div class="alert alert-block alert-warning">
    
**Question:** What are the differences in the two images?  How does bias subtracting, dark subtracting, and flat fielding our images help?

**Answer:** 

##  7.2: Now it is your turn.  Calibrate your cluster data following the same steps we did above.

<div class="alert alert-block alert-warning">
    
Remember to read all comments in the cells!  There are several hints for how to run each cell.

In [ ]:
# fill in the name of your cluster
cluster = '??'

### 1) Glob together your cluster images

In [ ]:
# First get the path and glob together our science frames

cluster_datadir = datadir + cluster + '/'+band+'band/'

# glob all files in sci_datadir
cluster_files = glob.glob()

print(band+' band science images')
print(np.array(cluster_files))

### 2) bias subtract all your cluster images.

In [ ]:
# We will first remove any existing bias subtracted frames.
b_clusterdata = glob.glob(datadir + cluster + '/'+band+'band/b*.fit*')
for im in b_clusterdata:
    os.remove(im)
    
cluster_files = glob.glob(datadir + cluster + '/'+band+'band/*.fit*')

# Bias subtract all the cluster data using our final calibration bias.
for fitsfile in cluster_files:
    # filename = fitsfile
    # path_to_bias = bias_path 
    # outpath = cluster + '/'+band+'band/'
    bias_subtract( )
         

### 3) Dark subtract the b_ files. But first, make a list of all the b_ cluster files using glob.

In [ ]:
# Glob your b_ cluster files

b_cluster_files = glob.glob(

print(band+' band bias subtracted science images')
print(np.array(b_cluster_files))

Dark subtract the cluster files.

In [ ]:
# We will first remove any existing dark subtracted frames.
db_clusterdata = glob.glob(datadir + cluster + '/'+band+'band/db*.fit*')
for im in db_clusterdata:
    os.remove(im)

print('dark subtracting cluster band')
for b_sci_image in b_cluster_files:
    # filename = b_sci_image
    # path_to_dark = dark_path 
    # outpath = cluster + '/'+band+'band/'

    #run the dark subtract function on your image.
    dark_subtract(
    

 ### 4) Flat field!

In [ ]:
# Glob your db_ cluster star files

# Fill in the ?? to glob the db_ cluster images.
db_cluster_files = glob.glob(cluster_datadir + '??_*.fit')


print(band+' band bias/dark subtracted standard star images')
print(np.array(db_cluster_files))

In [ ]:
# We will first remove any existing flat normalized frames.
fdb_clusterdata = glob.glob(datadir + cluster + '/'+band+'band/fdb*.fit*')
for im in fdb_clusterdata:
    os.remove(im)



# flat field the band db_ standard star frames.
print('flat fielding db_ cluster '+band+'band')
flat_data = fits.getdata(flat_path)
for db_sci_image in db_cluster_files:
    #open the data and header for the db_ image
    db_sci_data = fits.getdata(??)
    db_sci_hdr = fits.getheader(??)

    #divide the science image (db_sci_data) by the final calibration flat field (flat_data)
    fdb_sci_image = 
    
    #save the new flat fielded image!
    sci_name = db_sci_image.split('/')[-1]
    outpath = cluster + '/'+band+'band/'
    
    print('image saved to: ' + datadir + outpath + 'f' + sci_name)
    fits.writeto(datadir + outpath + 'f' + sci_name, fdb_sci_image, db_sci_hdr, overwrite=True )
        

## Congrats!  You have now calibrated your cluster data!  

As a reminder, for both the standard star and cluster data, you applied this equation to the raw images using the above python code:

$Final\ reduced\ image = \frac{ science\ image - bias -  dark \ (at\ same\ exposure\ time\ as\ science\ image)}{flat}$

Now, let's take a look at it.



In [ ]:
# plot a raw cluster frame and a final calibrated cluster frame.

#glob together all the flat field calibrated cluster data
final_cluster_files = glob.glob(datadir + cluster + '/'+band+'band/fdb*.fit')

#Now, use fits.getdata to read in a raw cluster file.
# replace ?? with a 0.
raw_cluster_file = fits.getdata(cluster_files[??])

# replace ?? with a 0.
final_cluster_file = fits.getdata(final_cluster_files[??])


fig, (ax1, ax2) = plt.subplots(figsize=(15,4), nrows=1, ncols=2)
vmin, vmax = interval.get_limits(??) #replace ?? with raw_cluster_file
im1 = ax1.imshow(??) #plot raw cluster file, replace ?? with raw_cluster_file and any other variables to make the plot pretty!
plt.colorbar(im1, ax=ax1)

vmin, vmax = interval.get_limits(??) #replace ?? with raw_cluster_file
im2 = ax2.imshow(??) #plot calibrated cluster file, replace ?? with raw_cluster_file and any other variables to make the plot pretty!
plt.colorbar(im2, ax=ax2)


#create a title, axis labels, change the colormap, to create a nice looking plot!

plt.show()


<div class="alert alert-block alert-danger">

**STOP:** Show your plot to the instructor!

<div class="alert alert-block alert-warning">
    
**Question:** What are the differences in the two images?  How does bias subtracting, dark subtracting, and flat fielding our images help?

**Answer:** 

BONUS: Open up your images in ds9 and play around with different scalings.  How does changing the scale affect the amount of noise you see?